# 基于字符的文本生成
给定一个字符，RNN自动根据该字符生成后续的文本.


*   此模型是基于字符的。训练开始时，模型不知道如何拼写一个英文单词，甚至不知道单词是文本的一个单位。
*   输出文本的结构类似于剧本 -- 文本块通常以讲话者的名字开始；而且与数据集类似，讲话者的名字采用全大写字母。
*   如下文所示，此模型由小批次 （batch） 文本训练而成（每批 100 个字符）。即便如此，此模型仍然能生成更长的文本序列，并且结构连贯。



In [1]:
# import reposity
from __future__ import absolute_import, division, print_function, unicode_literals

import tensorflow as tf

import numpy as np
import os
import time

In [2]:
# download dataset 
path_to_file = tf.keras.utils.get_file('shakespeare.txt', 'https://storage.googleapis.com/download.tensorflow.org/data/shakespeare.txt')
# load the dataset
text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
print(text[:100])

1122304/1115394 [==============================] - 0s 0us/step
First Citizen:
Before we proceed any further, hear me speak.

All:
Speak, speak.

First Citizen:
You


In [3]:
vocab = set(text)
vocab = sorted(vocab)
print(vocab)
# print vocab之后可以发现莎士比亚写的文章里面字符其实没有多少23333

['\n', ' ', '!', '$', '&', "'", ',', '-', '.', '3', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z']


In [4]:
# 创建从vocab到其索引的映射
char2idx = {u:i for i, u in enumerate(vocab)}
print("索引映射：")
print(char2idx, '\n')

idx2char = np.array(vocab)
print(idx2char,'\n')

# 创建了index和char的映射，然后可以将整个text转化为由index数字代表。
text_as_int = np.array([char2idx[c] for c in text])
print("转化为int的text文档前100个字符:")
print(text_as_int[:100])

索引映射：
{'\n': 0, ' ': 1, '!': 2, '$': 3, '&': 4, "'": 5, ',': 6, '-': 7, '.': 8, '3': 9, ':': 10, ';': 11, '?': 12, 'A': 13, 'B': 14, 'C': 15, 'D': 16, 'E': 17, 'F': 18, 'G': 19, 'H': 20, 'I': 21, 'J': 22, 'K': 23, 'L': 24, 'M': 25, 'N': 26, 'O': 27, 'P': 28, 'Q': 29, 'R': 30, 'S': 31, 'T': 32, 'U': 33, 'V': 34, 'W': 35, 'X': 36, 'Y': 37, 'Z': 38, 'a': 39, 'b': 40, 'c': 41, 'd': 42, 'e': 43, 'f': 44, 'g': 45, 'h': 46, 'i': 47, 'j': 48, 'k': 49, 'l': 50, 'm': 51, 'n': 52, 'o': 53, 'p': 54, 'q': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64} 

['\n' ' ' '!' '$' '&' "'" ',' '-' '.' '3' ':' ';' '?' 'A' 'B' 'C' 'D' 'E'
 'F' 'G' 'H' 'I' 'J' 'K' 'L' 'M' 'N' 'O' 'P' 'Q' 'R' 'S' 'T' 'U' 'V' 'W'
 'X' 'Y' 'Z' 'a' 'b' 'c' 'd' 'e' 'f' 'g' 'h' 'i' 'j' 'k' 'l' 'm' 'n' 'o'
 'p' 'q' 'r' 's' 't' 'u' 'v' 'w' 'x' 'y' 'z'] 

转化为int的text文档前100个字符:
[18 47 56 57 58  1 15 47 58 47 64 43 52 10  0 14 43 44 53 56 43  1 61 43
  1 54 56 53 41 43 43 42  1 39 52 63  1 44 59 56 58 

## 构造模型

使用RNN训练，每次输入长度为100的字符，以此的训练结果来预测：当给定一个字符时，下一个字符是什么。

In [11]:
# 设定输入长度
seq_length = 100

examples_per_epoch = len(text) // seq_length 

char_dataset = tf.data.Dataset.from_tensor_slices(text_as_int)

print(char_dataset)

<TensorSliceDataset shapes: (), types: tf.int64>


join: str.join(sequence)，将sequence序列中元素以str为间隔生成新字符串。

repr: 返回一个对象的 string 格式。

In [13]:
# batch 函数，把Dataset切分成seq_length大小的部分，drop_remainder代表了是否drop最后多余部分
sequences = char_dataset.batch(seq_length+1, drop_remainder = True)
for item in sequences.take(5):
  print(repr(''.join(idx2char[item.numpy()])))

'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '
'are all resolved rather to die than to famish?\n\nAll:\nResolved. resolved.\n\nFirst Citizen:\nFirst, you k'
"now Caius Marcius is chief enemy to the people.\n\nAll:\nWe know't, we know't.\n\nFirst Citizen:\nLet us ki"
"ll him, and we'll have corn at our own price.\nIs't a verdict?\n\nAll:\nNo more talking on't; let it be d"
'one: away, away!\n\nSecond Citizen:\nOne word, good citizens.\n\nFirst Citizen:\nWe are accounted poor citi'


In [15]:
def split_input_target(text):
  input_text = text[:-1]
  target_text = text[1:]
  return input_text, target_text

# input_text 输入数据包含整个字符串
# target_text 输出目标不包括第一个字符，这是和该问题的设计相关的，
# 问题设计中，要求输入一个字符，生成由此字符衍生的一系列字符，生成的部分是不包括事先输入的首字符的，因此target_text中不包括。

dataset = sequences.map(split_input_target)
# show dataset
for input_example, target_example in  dataset.take(1):
  print ('Input data: ', repr(''.join(idx2char[input_example.numpy()])))
  print ('Target data:', repr(''.join(idx2char[target_example.numpy()])))

Input data:  'First Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou'
Target data: 'irst Citizen:\nBefore we proceed any further, hear me speak.\n\nAll:\nSpeak, speak.\n\nFirst Citizen:\nYou '


In [17]:
BATCH_SIZE = 64

BUFFER_SIZE = 10000

dataset = dataset.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)

print(dataset)

<BatchDataset shapes: ((64, 64, 100), (64, 64, 100)), types: (tf.int64, tf.int64)>


# 创建模型


1.   使用了字词嵌入，word_embedding, 其将每个字符的数字映射到一个embedding_dim维度的向量中。
2.   GRU，简化版的RNN类型，大小由units=rnn_units制定（教程说用LSTM也可以）
3.   Dense 全连接层



In [19]:
vocab_size = len(vocab)
embedding_dim = 256
rnn_units = 1024

def build_model(vocab_size, embedding_dim, rnn_units, batch_size):
  model = tf.keras.Sequential([
    # 三层网络
    # 第一层embedding，对于每个字符，生成一个embedding，
    # 第二层GRU（RNN），把embedding输入到RNN，rnn_units定义为1024,
    # 第三层Dense 输入字符预测.
    tf.keras.layers.Embedding(vocab_size, embedding_dim, batch_input_shape=[batch_size, None]),
    tf.keras.layers.GRU(rnn_units, return_sequences=True,stateful=True,recurrent_initializer='glorot_uniform'),
    tf.keras.layers.Dense(vocab_size)
  ])
  model.summary()
  return model

# invoke build_model 
model = build_model(
    vocab_size = len(vocab),
    embedding_dim = embedding_dim,
    rnn_units=rnn_units,
    batch_size = BATCH_SIZE
)

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           16640     
_________________________________________________________________
gru (GRU)                    (64, None, 1024)          3938304   
_________________________________________________________________
dense (Dense)                (64, None, 65)            66625     
Total params: 4,021,569
Trainable params: 4,021,569
Non-trainable params: 0
_________________________________________________________________
